# Capstone project Week 3 assignment

##  Part 2
#### Includes part 1

### Import required libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge lxml --yes 
!pip install et_xmlfile

#!conda install -c conda-forge folium=0.11.0 --yes #installed 19jun20
import folium # plotting library


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



#### Read the table from the Wikipedia page directly by pandas

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = pd.read_html(url)

wikitable

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                           Neighborhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

#### It can be seen from the html output that the postal code table is in the [0] position.
#### A pandas data frame is created from position [0] of wikitable. 

In [3]:
df_postalCode=pd.DataFrame(wikitable[0])

df_postalCode

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Remove the boroughs that are "Not Assigned" and check if there will be any "NaN" neighborhood remaining.

In [5]:
notAssignedB = df_postalCode['Borough'] == 'Not assigned'
notAssignedN = df_postalCode['Neighborhood'] == 'Not assigned'
notAssignedIndexB= df_postalCode[notAssignedB].index
notAssignedIndexN= df_postalCode[notAssignedN].index
if all(notAssignedIndexB == notAssignedIndexN):
    df_postalCode.drop(notAssignedIndexB, inplace=True)
    print("None unassigned neighborhoods remain")

None unassigned neighborhoods remain


#### There are no neighborhoods to be renamed after removing the "Not Assigned" boroughs.

In [6]:
print(df_postalCode.shape)
df_postalCode

(103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### The resulting data frame has 103 postal codes (rows).
#### Checking if any Postal Code is repeated; if yes, the corresponding neighborhoods will need to be merged.

In [7]:
df_postalCode.nunique()

Postal Code     103
Borough          10
Neighborhood     99
dtype: int64

#### There are 103 different postal codes, which means there are no repeats.
#### There are ten different boroughs in the 103 postal codes. It was known in advance that boroughs are repeated.
#### There are four repeated neighborhoods. Which ones?

In [8]:
dups=df_postalCode['Neighborhood'].duplicated()
df_postalCode[dups].sort_values(by=df_postalCode.columns.to_list())

,Postal Code,Borough,Neighborhood
20,M3C,North York,Don Mills
74,M3L,North York,Downsview
83,M3M,North York,Downsview
92,M3N,North York,Downsview


In [9]:
dups2 = df_postalCode['Neighborhood'] == 'Don Mills'
dups3 = df_postalCode['Neighborhood'] == 'Downsview'
print(df_postalCode[dups2].sort_values(by=df_postalCode.columns.to_list()),'\n')
print(df_postalCode[dups3].sort_values(by=df_postalCode.columns.to_list()))

   Postal Code     Borough Neighborhood
11         M3B  North York    Don Mills
20         M3C  North York    Don Mills 

   Postal Code     Borough Neighborhood
65         M3K  North York    Downsview
74         M3L  North York    Downsview
83         M3M  North York    Downsview
92         M3N  North York    Downsview


#### According to the repeats lists, Don Mills appears twice in the data frame and Downsview appears four times.
#### However, there is indeed a different postal code for every repeated neighborhood.
#### This means that there is nothing more to clean up in this data set and there are 103 different postal codes in Toronto with an assigned borough and neighborhood.

In [10]:
print(df_postalCode.shape)

df_postalCode

(103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Part 2

#### Install and Import required library

In [11]:
!pip install pgeocode
import pgeocode

#### Set country to query as Canada ('ca').
#### Create a list 'l_postalCode' of the postal codes from the 'df_postalCode' data frame.
#### Loop over 'l_postalCode' to obtain the geo data for each postal code as a data frame 'df_latlon'.
#### Check the columns of 'df_latlon' to identify where the latitute and logitude are.

In [12]:
nomi = pgeocode.Nominatim('ca')
l_postalCode = df_postalCode['Postal Code'].tolist()
df_latlon = pd.DataFrame()
for i in l_postalCode:
    df_latlon = df_latlon.append (nomi.query_postal_code([i]))

df_latlon.columns

Index(['postal_code', 'country code', 'place_name', 'state_name', 'state_code',
       'county_name', 'county_code', 'community_name', 'community_code',
       'latitude', 'longitude', 'accuracy'],
      dtype='object')

#### Latitiude and Longitude are in columns 'latitude' and 'longitude' of 'df_latlon' respectively.
#### Columns for latitude and longitude are added to 'df_postalCode' data frame from 'df_latlon'.
#### This completes the data frame format required.

In [13]:
df_postalCode['latitude'] = df_latlon['latitude'].tolist()
df_postalCode['longitude'] = df_latlon['longitude'].tolist()

df_postalCode

,Postal Code,Borough,Neighborhood,latitude,longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
165,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


### Check that there is no missing values for latitude and/or longitude.

In [14]:
NAlon = df_postalCode['longitude'].astype('str') == 'nan'
NAlat = df_postalCode['latitude'].astype('str') == 'nan'
NAlonIndex = df_postalCode[NAlon].index
NAlatIndex = df_postalCode[NAlat].index
if all(NAlonIndex == NAlatIndex):
    print(df_postalCode.loc[NAlatIndex])
else:
    print(df_postalCode.loc(NAlatIndex))
    print(df_postalCode.loc(NAlonIndex))

    Postal Code      Borough                           Neighborhood  latitude  \
114         M7R  Mississauga  Canada Post Gateway Processing Centre       NaN   

     longitude  
114        NaN  


### The borough Mississauga with postal code M7R does not have the latitude-longitude information.
### This information will be obtained using geopy with the borough as address.

In [15]:
address = 'Mississauga, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude2 = location.latitude
longitude2 = location.longitude
print('The coordinates of Mississauga are {}, {}.'.format(latitude2, longitude2))

The coordinates of Mississauga are 43.590338, -79.645729.


### These will be added to the data frame 'df_postalCode' at the corresponding position (114).

In [16]:
df_postalCode.loc[NAlatIndex, 'latitude'] = latitude2
df_postalCode.loc[NAlatIndex, 'longitude'] = longitude2

df_postalCode.loc[NAlonIndex]

,Postal Code,Borough,Neighborhood,latitude,longitude
114,M7R,Mississauga,Canada Post Gateway Processing Centre,43.590338,-79.645729


### The data frame is now complete (no missing values) in the required format.

In [17]:
df_postalCode

,Postal Code,Borough,Neighborhood,latitude,longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
165,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939
